# Initialization

In [64]:
import glob
import pandas as pd
import numpy as np
import datetime as dt
import os
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# import nltk libraries for text mining
import re
import string
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

## Sentiment
from nltk.corpus import twitter_samples
from nltk.tokenize import TweetTokenizer

import string
import re

from nltk.corpus import stopwords 
stopwords_english = stopwords.words('english')

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Path

In [65]:
DATA_PATH = "D:\\OneDrive - National University of Singapore\\NUS MTech KE\\MTech KE - FYP - InsureSense\\Kang Jiang\\System Implementation\\Data Acquisition and Storage\\Twitter\\output\\"

# Defined Function

In [66]:
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [67]:
def import_data(folder_name):
    # Japan flood
    # Typhoon Jebi
    # Typhoon Mangkhut

    path = DATA_PATH + folder_name
    allFiles = glob.glob(path + "\\*.csv")

    df_list = []
    for file in allFiles:
        df = pd.read_csv(file, header=0, engine='python').iloc[:, 1:]
        df_list.append(df)


    df = pd.concat(df_list).reset_index(drop=True)
    df = df[['user','timestamp','text','likes','replies','retweets','tweet_id','url']]
    
    return df

In [68]:
# # define a function for text mining with the following steps:
# # 1. remove the non English words
# # 2. tokenize the string for each row
# # 3. remove punctiation
# # 4. convert each of the token to lower case
# # 5. remove stopwords
# # 6. lemmatize each of the token
# # 7. join the tokens back into string

# mystopwords = stopwords.words("English")
# wnlemma = nltk.WordNetLemmatizer()
# def text_process(text):
#     text = re.sub(r'\d+', '', text)
#     tokens = nltk.word_tokenize(text)
#     tokens_nop = [word for word in tokens if word not in string.punctuation]
#     tokens_lower = [ word.lower() for word in tokens_nop ]
#     tokens_nostop = [word for word in tokens_lower if word not in mystopwords]
#     tokens_lemma = [wnlemma.lemmatize(word) for word in tokens_nostop]
#     text_after_process = " ".join(tokens_lemma)
#     return(text_after_process)

In [69]:
def clean_data(dataframe, events):
    df = dataframe.dropna(subset=['text'])
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y-%m-%d %H:%M:%S")
    df['date'] = df['timestamp'].dt.date
    df['events'] = events
#     df['text_processed'] = df['text'].apply(text_process)

#     length = df['text'].apply(len)
#     df = df.assign(length_text=length)

#     length = df['text_processed'].apply(len)
#     df = df.assign(length_text_processed=length)

#     df = df.drop_duplicates(subset=['text'], keep='first')
#     df = df.drop_duplicates(subset=['text_processed'], keep='first')
    
#     df = df[['user','timestamp','date','search_terms','text','text_processed','length_text','length_text_processed','likes','replies','retweets','tweet_id','url']]
    df = df[['tweet_id','user','timestamp','date','events','text','likes','replies','retweets','url']]
    df = df.sort_values(by = ['date'], ascending = True)
    df = df.reset_index(drop=True)
    return df

In [70]:
def clean_tweets(tweet):

    wnlemma = nltk.WordNetLemmatizer()
    printable = set(string.printable)

    # remove non ASCII word
    tweet = ''.join(filter(lambda x: x in printable, tweet))
    
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)

    tweet = tweet.strip()
    
    tweet = re.sub(' +',' ', tweet)
    
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_english and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            lemma_word = wnlemma.lemmatize(word)
            tweets_clean.append(lemma_word)
            
    tweets_clean = " ".join(tweets_clean)
    return tweets_clean

# Data Cleansing and Transformation

In [71]:
# Japan Floods 1
# Japan Floods 2
# Typhoon Jebi 1
# Typhoon Jebi 2
# Typhoon Mangkhut 1
# Typhoon Mangkhut 2

In [72]:
df_Japan_Floods_1 = import_data('Japan Floods 1')
df_Japan_Floods_2 = import_data('Japan Floods 2')

df_Typhoon_Jebi_1 = import_data('Typhoon Jebi 1')
df_Typhoon_Jebi_2 = import_data('Typhoon Jebi 2')

df_Typhoon_Mangkhut_1 = import_data('Typhoon Mangkhut 1')
df_Typhoon_Mangkhut_2 = import_data('Typhoon Mangkhut 2')

In [73]:
frames = [df_Japan_Floods_1, df_Japan_Floods_2]
df_Japan_Floods_full = pd.concat(frames)
df_Japan_Floods_full = df_Japan_Floods_full.reset_index(drop=True)

frames = [df_Typhoon_Jebi_1, df_Typhoon_Jebi_2]
df_Typhoon_Jebi_full = pd.concat(frames)
df_Typhoon_Jebi_full = df_Typhoon_Jebi_full.reset_index(drop=True)

frames = [df_Typhoon_Mangkhut_1, df_Typhoon_Mangkhut_2]
df_Typhoon_Mangkhut_full = pd.concat(frames)
df_Typhoon_Mangkhut_full = df_Typhoon_Mangkhut_full.reset_index(drop=True)

In [74]:
df_Japan_Floods_full.info()
# df_Japan_Floods_full = df_Japan_Floods_full[:100]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81335 entries, 0 to 81334
Data columns (total 8 columns):
user         81335 non-null object
timestamp    81335 non-null object
text         81335 non-null object
likes        81335 non-null int64
replies      81335 non-null int64
retweets     81335 non-null int64
tweet_id     81335 non-null int64
url          81335 non-null object
dtypes: int64(4), object(4)
memory usage: 5.0+ MB


In [75]:
df_Typhoon_Jebi_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88160 entries, 0 to 88159
Data columns (total 8 columns):
user         88160 non-null object
timestamp    88160 non-null object
text         88160 non-null object
likes        88160 non-null object
replies      88160 non-null object
retweets     88160 non-null object
tweet_id     88160 non-null object
url          88160 non-null object
dtypes: object(8)
memory usage: 5.4+ MB


In [76]:
df_Typhoon_Mangkhut_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71058 entries, 0 to 71057
Data columns (total 8 columns):
user         71050 non-null object
timestamp    71050 non-null object
text         71052 non-null object
likes        71054 non-null float64
replies      71054 non-null object
retweets     71054 non-null object
tweet_id     71050 non-null float64
url          71050 non-null object
dtypes: float64(2), object(6)
memory usage: 4.3+ MB


In [77]:
df_Japan_Floods_full = clean_data(df_Japan_Floods_full, 'Japan Floods')
df_Japan_Floods_full = df_Japan_Floods_full.reset_index(drop=True)

In [78]:
df_Typhoon_Jebi_full = clean_data(df_Typhoon_Jebi_full, 'Typhoon Jebi')
df_Typhoon_Jebi_full = df_Typhoon_Jebi_full.reset_index(drop=True)

In [79]:
df_Typhoon_Mangkhut_full = clean_data(df_Typhoon_Mangkhut_full, 'Typhoon Mangkhut')
df_Typhoon_Mangkhut_full = df_Typhoon_Mangkhut_full.reset_index(drop=True)

C:\Users\purpl\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\purpl\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\purpl\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

In [80]:
df_Japan_Floods_full['processed_text'] = ""
for index, row in df_Japan_Floods_full.iterrows():
    df_Japan_Floods_full['processed_text'].iloc[index] = clean_tweets(row['text'])
df_Japan_Floods_full = df_Japan_Floods_full[['tweet_id','user','timestamp','date','events','text','processed_text','likes','replies','retweets','url']]

C:\Users\purpl\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [81]:
df_Typhoon_Jebi_full['processed_text'] = ""
for index, row in df_Typhoon_Jebi_full.iterrows():
    df_Typhoon_Jebi_full['processed_text'].iloc[index] = clean_tweets(row['text'])
df_Typhoon_Jebi_full = df_Typhoon_Jebi_full[['tweet_id','user','timestamp','date','events','text','processed_text','likes','replies','retweets','url']]

C:\Users\purpl\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [82]:
df_Typhoon_Mangkhut_full['processed_text'] = ""
for index, row in df_Typhoon_Mangkhut_full.iterrows():
    df_Typhoon_Mangkhut_full['processed_text'].iloc[index] = clean_tweets(row['text'])
df_Typhoon_Mangkhut_full = df_Typhoon_Mangkhut_full[['tweet_id','user','timestamp','date','events','text','processed_text','likes','replies','retweets','url']]

C:\Users\purpl\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [83]:
df_Japan_Floods_full.head(5)

,tweet_id,user,timestamp,date,events,text,processed_text,likes,replies,retweets,url
0,1005599354780958720,@Invesment_JpnJp,2018-06-09 23:55:43,2018-06-09,Japan Floods,We help you with Japan Tax Return at a reasona...,help japan tax return reasonable price minimum...,0,0,0,/Invesment_JpnJp/status/1005599354780958720
1,1005353978069209088,@TokyoAdultGuide,2018-06-09 07:40:41,2018-06-09,Japan Floods,New entry in Q&A has been added... Choosing be...,new entry q added ... choosing threesome two h...,1,0,0,/TokyoAdultGuide/status/1005353978069209088
2,1005354512142553088,@KoltovskoyYakov,2018-06-09 07:42:48,2018-06-09,Japan Floods,Baghdad blasts kill 18 after recount law OKâ€™...,baghdad blast kill 18 recount law okd japan news,0,0,0,/KoltovskoyYakov/status/1005354512142553088
3,1005354564428681216,@CherryMinus,2018-06-09 07:43:01,2018-06-09,Japan Floods,"All this C94 preparation, I'm excited... Even ...",c94 preparation i'm excited ... even though i'...,0,0,0,/CherryMinus/status/1005354564428681216
4,1005355099541999619,@shinryutaishi,2018-06-09 07:45:08,2018-06-09,Japan Floods,Save our Nippon.\nSave our Japan.\nSave our co...,save nippon save japan save country themipn,0,0,0,/shinryutaishi/status/1005355099541999619


In [84]:
df_Typhoon_Jebi_full.head(5)

,tweet_id,user,timestamp,date,events,text,processed_text,likes,replies,retweets,url
0,1022632341351669760,@EqualizerSoccer,2018-07-26 23:58:44,2018-07-26,Typhoon Jebi,42' - Ertz gets wide open inside Japan's box b...,42 ertz get wide open inside japan's box keepe...,7,0,1,/EqualizerSoccer/status/1022632341351669760
1,1022400668013744128,@pedromj,2018-07-26 08:38:09,2018-07-26,Typhoon Jebi,Typhoon Jongdari remains on course to hit Japa...,typhoon jongdari remains course hit japan's ma...,0,0,0,/pedromj/status/1022400668013744128
2,1022400406066872320,@tokyostyle_no1,2018-07-26 08:37:06,2018-07-26,Typhoon Jebi,I'm Satomi\nYou can shake off the fatigue of t...,i'm satomi shake fatigue traveling japan happy...,0,0,0,/tokyostyle_no1/status/1022400406066872320
3,1022400138122211328,@HongKongFP,2018-07-26 08:36:02,2018-07-26,Typhoon Jebi,Palau seeks help from US and Japan to counter ...,palau seek help u japan counter china's touris...,5,1,3,/HongKongFP/status/1022400138122211328
4,1022399914645639173,@dwnreport,2018-07-26 08:35:09,2018-07-26,Typhoon Jebi,Palau asks US and Japan for help after China i...,palau asks u japan help china imposes tourist ...,0,0,0,/dwnreport/status/1022399914645639173


In [85]:
df_Typhoon_Mangkhut_full.head(5)

,tweet_id,user,timestamp,date,events,text,processed_text,likes,replies,retweets,url
0,1.026975e+18,@Kuwago68,2018-08-07 23:35:23,2018-08-07,Typhoon Mangkhut,Ummm...Well...Ummm...To Late. He'd already mad...,ummm ... well ... ummm ... late he'd already m...,1.0,1,0,/Kuwago68/status/1026975117950107648
1,1.026760e+18,@SCMPNews,2018-08-07 09:20:03,2018-08-07,Typhoon Mangkhut,Crowdfunding to help needy Hong Kong cancer pa...,crowdfunding help needy hong kong cancer patie...,3.0,0,2,/SCMPNews/status/1026759867103621120
2,1.026759e+18,@jtmama,2018-08-07 09:17:10,2018-08-07,Typhoon Mangkhut,U gent help needed..... please contact either ...,u gent help needed ... please contact either u...,0.0,0,0,/jtmama/status/1026759142696841216
3,1.026756e+18,@fidel_hon,2018-08-07 09:04:41,2018-08-07,Typhoon Mangkhut,Im from Bloody Hong Kong so i should kill myself,im bloody hong kong kill,0.0,0,0,/fidel_hon/status/1026755999363883008
4,1.026746e+18,@IFRAsia,2018-08-07 08:25:11,2018-08-07,Typhoon Mangkhut,China Everbright Water seeks dual primary list...,china everbright water seek dual primary listi...,0.0,0,0,/IFRAsia/status/1026746061023535104


# Data Storage

In [86]:
df_Japan_Floods_full.to_csv(DATA_PATH + "\\Full\\df_Japan_Floods_full.csv")
df_Typhoon_Jebi_full.to_csv(DATA_PATH + "\\Full\\df_Typhoon_Jebi_full.csv")
df_Typhoon_Mangkhut_full.to_csv(DATA_PATH + "\\Full\\df_Typhoon_Mangkhut_full.csv")